TODO for next round of submissions:
- submit 5c (shuffled training) to compare performance with current submission (5b)
- do hyperparameter optimization via grid search

Factors that can influence taxi fare:
- Distance
- Travel time
- Time of day
- Airport start
- Airport finish
- West/southbound bridge crossing

In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import shapely as shp

from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing

import time
from datetime import datetime, timedelta

from pyproj import Proj
import geopy
from geopy.geocoders import Nominatim

import folium
from IPython.display import HTML

import json
import urllib

import random

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
seed = 0

In [3]:
def load_data(fname,modified=True,end_dates=True,shuffle=True):
    names=['ID', 'start_date', 'end_date', 'fare', 'passengers',
            'start_lo', 'start_la', 'end_lo', 'end_la',
            'start_TAZ', 'end_TAZ']
    np.random.seed(seed)
    if modified:
        if end_dates:
            data = pd.read_csv(
                fname,
                skiprows=1,
                names=names+['dist','timecost','distcost'],
                index_col=0,
                parse_dates=['start_date','end_date'])
        else:
            data = pd.read_csv(
                fname,
                skiprows=1,
                names=names+['dist','timecost','distcost'],
                index_col=0,
                parse_dates=['start_date'])
            
        if shuffle:
            # shuffle indices
            new_ix = list(data.index)
            np.random.shuffle(new_ix)
            data = data.reindex(index=new_ix)
    
    # deprecated, since I'm definitely going to use the modified dataset
    else:
        if end_dates:
            data = pd.read_csv(
                fname,
                names=names,
                index_col=0,
                parse_dates=['start_date','end_date'])
        else:
            data = pd.read_csv(
                fname,
                names=names,
                index_col=0,
                parse_dates=['start_date'])
            # shuffle indices
            new_ix = list(data.index)
            np.random.shuffle(new_ix)
            data = data.reindex(index=new_ix)
    return data

## Exploratory DA

### Load data

In [4]:
data = load_data('Taxi_Train_Routed.csv',shuffle=False)

### Clean data and subset for visual EDA

In [5]:
# don't remove 0-passenger outlier (not using passengers in model, so will leave in)
# data = data[data['passengers'] != 0]

# drop bad data
data_clean = data
# remove 0-fare data and implausibly low fares
data_clean = data[data['fare'] > 3.5]
# drop out-of-area low fare from Livermore area?
data_clean = data_clean[data_clean['start_lo'] <= -122.197255]
# no missing lat/long data (good!)

# get index for the kept data
ix_clean = data_clean.index



# subset data for folium loading
# subset = data.sample(n=5000,random_state=seed)
# subset = data.iloc[10000:20000,:]
# subset = data[data['start_la'] > (data.min()['start_la'] + (data.max()['start_la'] - data.min()['start_la'])*.7)]
# subset = data[data['fare'] < 4]
# coords = subset[['start_lo','start_la','end_lo','end_la','start_TAZ','end_TAZ']]

### First, visualize the origin and destination, to see what region this dataset encompases. Look at potential bridge tolls, airport fees as contribution

In [ ]:
# use folium to map
center = list(coords[['start_la','start_lo']].mean())

def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

map_osm = folium.Map(location=center,zoom_start=9)
for i in range(0,coords.shape[0]):
    marker_data = coords.iloc[i,:]
    map_osm.simple_marker([marker_data['start_la'],marker_data['start_lo']],popup='Start'+str(i))
    map_osm.simple_marker([marker_data['end_la'],marker_data['end_lo']],popup='End'+str(i))
inline_map(map_osm)



Results: the majority of fares are within the city, but there are starting points outside of the city & at airports. Should include indicator for airport start, airport end, west/southbound bridge crossing

### Look at relationship of n_passengers to fare - shouldn't be one...

In [ ]:
data.plot(kind='scatter',x='passengers',y='fare')

Looks like it could have an effect, but I bet it will disappear when we take into account distance traveled. There was a 0-passenger data point, but I'm going to remove him (bad data)

## Preproccessing

### Generate and append calculated features

In [6]:
## TRAVEL TIME
# convert to seconds since midnight
to = (data['start_date'] - data['start_date'].apply(pd.datetools.normalize_date)).dt.seconds

# compute tdelta
tdelta = (data['end_date'] - data['start_date']).dt.seconds


## TRAVEL DIST
# convert to meters in UTM
myProj = Proj("+proj=utm +zone=10 +ellps=WGS84 +datum=WGS84")
o_x,o_y = myProj(data['start_lo'].values,data['start_la'].values)
d_x,d_y = myProj(data['end_lo'].values,data['end_la'].values)

# compute l-2 and l-1 norm (l-1 potentially more useful as it describes driving distance if roads are all N-S...)
d_l2 = np.sqrt((d_x-o_x)**2 + (d_y-o_y)**2)
d_l1 = (d_x-o_x) + (d_y-o_y)


## AIRPORT DEPARTURES AND ARRIVALS
# TAZ info
    # SFO: 239
    # OAK: 895
o_sfo,d_sfo,o_oak,d_oak = data['start_TAZ']==239, data['end_TAZ']==239, data['start_TAZ']==895, data['end_TAZ']==895


## TOLL LIKELIHOOD
# generate "as the crow flies" line denoting route
route_line = gp.GeoSeries(
    [shp.geometry.LineString([(o_lo,o_la),(d_lo,d_la)]) for o_lo,o_la,d_lo,d_la in zip(
            data['start_lo'],
            data['start_la'],
            data['end_lo'],
            data['end_la'])],
    index=data.index)

# create line separating east bay and SF
dumbarton_bottom = [-122.06771850585938,37.46705828325743]
dumbarton_top = [-122.16384887695312,37.54658486885501]
sanMateo_top = [-122.28744506835938,37.69234455983933]
bay_top = [-122.35954284667969,37.884439502831555]
richmond_top = [-122.49886265167493,38.04134704836284]

gg_E = [-122.420654296875,37.83131066825093]
gg_W = [-122.94387817382812,37.68038998034691]

dumbarton_sep = shp.geometry.LineString([dumbarton_bottom,dumbarton_top])
sanMateo_sep = shp.geometry.LineString([dumbarton_top,sanMateo_top])
bay_sep = shp.geometry.LineString([sanMateo_top,bay_top])
richmond_sep = shp.geometry.LineString([bay_top,richmond_top])
gg_sep = shp.geometry.LineString([gg_E,gg_W])

# create indicators if route likely crossed each bridge going S/W
toll_d = route_line.intersects(dumbarton_sep) & (data['start_lo'] > data['end_lo'])
toll_s = route_line.intersects(sanMateo_sep) & (data['start_lo'] > data['end_lo'])
toll_b = route_line.intersects(bay_sep) & (data['start_lo'] > data['end_lo'])
toll_r = route_line.intersects(richmond_sep) & (data['start_lo'] > data['end_lo'])
toll_g = route_line.intersects(gg_sep) & (data['start_la'] > data['end_la'])

# only the bay bridge has a few toll payers (5), and a few of these look like they could have mistakes.
# Including as a potentially useful feature
toll = toll_b

### Generate full datasets

In [7]:
X_full = pd.DataFrame({
        'to':to,
        'd_l1':d_l1,
        'd_l2':d_l2,
        'd_pg':data['distcost'],
        'o_sfo':o_sfo,
        'd_sfo':d_sfo,
        'o_oak':o_oak,
        'd_oak':d_oak,
        'start_TAZ':data['start_TAZ'],
        'end_TAZ':data['end_TAZ'],
        'toll':toll,
        'tdelta_pg':data['timecost'],
        'passengers':data['passengers']
    })
X_full = X_full.reindex(columns=['to','d_l1','d_l2','d_pg','o_sfo',
        'd_sfo','o_oak','d_oak','start_TAZ','end_TAZ','toll','tdelta_pg','passengers'])
T = pd.Series(tdelta)
Y = data['fare']

In [8]:
scalar = None

# If we don't want all distance calculations included
# time_cols = ['to','d_pg','o_sfo','d_sfo','o_oak','d_oak','start_TAZ','end_TAZ','toll','tdelta_pg']

# include passengers? (doesn't seem to help)
# time_cols = ['to','d_pg','o_sfo','d_sfo','o_oak','d_oak','start_TAZ','end_TAZ','toll','tdelta_pg','passengers']

# settled on these for features to determine time of trip
time_cols = ['to','d_pg','start_TAZ','end_TAZ','d_l1','tdelta_pg']
# time_cols = ['to','start_TAZ'] # specifically to create profitability heat map by time of day

# starting from basics for features to determine cost of trip
fare_cols = ['to','d_pg','d_l1','tdelta_pred','start_TAZ','end_TAZ','tdelta_pg','toll']
# fare_cols = ['to','start_TAZ'] # specifically to create profitability heat map by time of day

X = X_full.reindex(columns=time_cols)
X_fare = X_full.reindex(columns=fare_cols)

# Try dummifying categoricals (didn't seem to help and way less efficient)
# X = pd.get_dummies(X,columns=['start_TAZ','end_TAZ'])

# get clean dataset for training
X_clean = X.reindex(index=ix_clean)
T_clean = T.reindex(index=ix_clean)
Y_clean = Y.reindex(index=ix_clean)

# scale? (unclear if it helps)
# scalar = preprocessing.StandardScaler().fit(X_clean)
# X = scalar.transform(X)
# X_clean = scalar.transform(X_clean)

# scale only the continuous vars?
# non_cont_vars = X.reindex(columns=['o_sfo','d_sfo','o_oak','d_oak','start_TAZ','end_TAZ','toll'])
# cont_vars = X.reindex(columns=['to','d_pg','tdelta_pg'])
# scalar = preprocessing.StandardScaler().fit(cont_vars)
# cont_vars_scaled = scalar.transform(cont_vars)
# X = np.hstack((non_cont_vars.as_matrix(),cont_vars_scaled))

## Train and Test Models

### Baseline model (predicts overal mean fare for all data)

In [9]:
mean_fare = Y.mean()
rmse_baseline = np.sqrt(((Y - mean_fare)**2).mean())
print 'Baseline RMSE:', rmse_baseline

Baseline RMSE: 16.2152982839


### First develop a model for tdelta

In [46]:
# random forest with pre-selected, default hyperparameters (using OOB score)
TTmodel = RandomForestRegressor(n_estimators=1000,random_state=seed,oob_score=True,max_features='sqrt',n_jobs=-1,
                               max_depth=13)
TTmodel.fit(X_clean,T_clean)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=True, random_state=0,
           verbose=0, warm_start=False)

In [47]:
# max_features = sqrt, max_depth=13 - 1000 trees
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.677455988846
[ 0.07906021  0.39059255  0.04862131  0.08174516  0.14549518  0.25448559]


In [45]:
# max_features = sqrt, max_depth=13 - 100 trees
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.67375924674
[ 0.0782447   0.4097142   0.04304123  0.08632155  0.14263487  0.24004345]


In [11]:
# max_features = sqrt - 1000 trees
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.66768015207
[ 0.11892483  0.33676027  0.05711293  0.08455217  0.17339237  0.22925744]


In [1340]:
# max_features = sqrt - 100 trees
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.659322316194
[ 0.11898633  0.35575389  0.05261577  0.08682394  0.1708041   0.21501597]


In [1338]:
# no max_features - 100 trees
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.652785044585
[ 0.1205508   0.65564546  0.03888415  0.03996914  0.09286139  0.05208907]


In [1187]:
# basics + d_l1 (5000 trees) - **CURRENT WINNER**
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.661042954435
[ 0.12097615  0.65585835  0.03887621  0.04025422  0.09282464  0.05121043]


In [1153]:
# basics + d_l1 (1000 trees)
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.660515095521
[ 0.12090801  0.65569451  0.03903179  0.04013464  0.09293356  0.05129749]


In [1170]:
# basics + d_l1 (1000 trees) + shuffled
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.659882249817
[ 0.12077517  0.65560884  0.03907315  0.04039182  0.09315811  0.05099291]


In [1137]:
# basics + d_l1 - *winner for small 10 trees*
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.526169439791
[ 0.12236231  0.65527683  0.03999273  0.03936146  0.08926985  0.05373682]


In [1126]:
# basics + d_l1, shuffled
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.516586518816
[ 0.12399812  0.65486359  0.03988887  0.03843279  0.09225773  0.05055892]


In [1020]:
# basics + d_l1 - TAZs
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.513565862271
[ 0.12890178  0.67600021  0.12214351  0.0729545 ]


In [1014]:
# adding toll (very small importance, and only affects a couple observations, so will leave out)
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.523555323411
[  1.74033767e-01   6.65108453e-01   4.80798008e-02   4.87803463e-02
   2.94852208e-05   6.39681478e-02]


In [1007]:
# smallest set of features
print TTmodel.oob_score_
print TTmodel.feature_importances_

0.522280117915
[ 0.17383775  0.66568788  0.04773342  0.04890291  0.06383804]


In [48]:
tdelta_pred = TTmodel.predict(X)

# Note the reindexing for the fare prediction IS NOT IMPLEMENTED FOR THE SCALED VERSION, SINCE I PRETTY 
# MUCH DECIDED NOT TO SCALE
if scalar:
    tdelta_pred_clean = TTmodel.predict(X_clean)
    tdelta_array_clean = np.array([tdelta_pred_clean]).T
    scalar_t = preprocessing.StandardScaler().fit(tdelta_array_clean)
    tdelta_scaled_clean = scalar_t.transform(tdelta_array_clean)
    X_clean = np.hstack((X_clean,tdelta_scaled_clean))
    
    tdelta_array = np.array([tdelta_pred]).T
    tdelta_scaled = scalar_t.transform(tdelta_array)
    X = np.hstack((X,tdelta_scaled))
else:
    X_fare['tdelta_pred']=tdelta_pred
    X_fare = X_fare.reindex(columns=fare_cols)
    # again,get clean dataset for training
    X_fare_clean = X_fare.reindex(index=ix_clean)

In [983]:
# # random forest Grid Search for optimal parameters
# tuned_params = {max_depth:[None,1,10,100]}
# if model_type == 'RF':
#             self.TTmodel = GridSearchCV(RandomForestRegressor,tuned_params, cv = cv, (random_state=0, n_estimators=50, max_depth=50)
#             self.TTmodel.fit(self.X, self.T)
#         else:
#             print 'Unknown model type for travel times predictor.'

### Now predict Fare

In [87]:
# random forest with pre-selected, default hyperparameters
Fmodel = RandomForestRegressor(n_estimators=1000,random_state=seed,oob_score=True,n_jobs=-1,max_features='sqrt',
                              max_depth=16)
Fmodel.fit(X_fare_clean,Y_clean)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=True, random_state=0,
           verbose=0, warm_start=False)

In [86]:
# max_features='sqrt', 1000 trees
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.93914638888
[  1.26400654e-02   3.29755943e-01   8.92724087e-02   2.00364244e-01
   4.91378621e-02   1.10184338e-01   2.08616262e-01   2.88764932e-05]


In [84]:
# no max_features, 1000 trees, max_depth=16
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.938174128449
[  8.47620937e-03   8.77066835e-01   8.28598596e-03   8.79384607e-02
   4.32116290e-03   6.90341377e-03   6.97227650e-03   3.56554372e-05]


In [82]:
# max_features=sqrt, 1000 trees, max_depth=16
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.939529426031
[  9.48921621e-03   3.31539350e-01   8.60255712e-02   2.02663556e-01
   4.78727155e-02   1.11250928e-01   2.11130384e-01   2.82791998e-05]


In [80]:
# max_features=sqrt, 100 trees, max_depth=16
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.939280998697
[  9.25154681e-03   2.86389990e-01   1.03238514e-01   2.13642194e-01
   3.84773741e-02   1.04481650e-01   2.44486271e-01   3.24607624e-05]


In [52]:
# max_features=sqrt, 100 trees
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.937515628586
[  1.30085533e-02   2.82253431e-01   1.08581899e-01   2.20012991e-01
   3.98599430e-02   9.69819224e-02   2.39280311e-01   2.09502157e-05]


In [50]:
# no max_features, 100 trees
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.937054538864
[  1.09176336e-02   8.69072497e-01   1.03189111e-02   8.93683359e-02
   4.95799011e-03   7.59829283e-03   7.72727515e-03   3.90639709e-05]


In [15]:
# same as below but unshuffled, max_depth = sqrt, and w/ 1000 trees
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.947921110285
[  1.12149493e-02   3.33812306e-01   8.88370890e-02   1.89909379e-01
   4.86011525e-02   1.14297787e-01   2.13295586e-01   3.17510021e-05]


In [1191]:
# same as below but unshuffled and w/ 5000 trees - **CURRENT WINNER**
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.949018959354
[  8.75633125e-03   8.70256997e-01   8.61004138e-03   9.48068729e-02
   3.91434876e-03   6.99165218e-03   6.61741859e-03   4.63375204e-05]


In [1156]:
# all features from time predition (basics + TAZs + PG time estimate + time of day at origin) + toll (1000 trees)
print Fmodel.oob_score_
print Fmodel.feature_importances_
['to','d_pg','d_l1','tdelta_pred','start_TAZ','end_TAZ','tdelta_pg','toll']

0.948905502461
[  8.70733198e-03   8.70163175e-01   8.64677801e-03   9.48703342e-02
   3.92550766e-03   7.00607524e-03   6.63835643e-03   4.24413604e-05]


In [1173]:
# all features from time predition (basics + TAZs + PG time estimate + time of day at origin) + toll (1000 trees) + shuffled
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.948620715911
[  8.72657517e-03   8.70071812e-01   8.68699414e-03   9.48803931e-02
   4.00916818e-03   6.82569163e-03   6.75465243e-03   4.47132532e-05]


In [1140]:
# all features from time predition (basics + TAZs + PG time estimate + time of day at origin) + toll - winner for 10 trees
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.912797695171
[  9.42865503e-03   8.70817062e-01   8.65432713e-03   9.28411175e-02
   4.31332236e-03   7.37650385e-03   6.48898176e-03   8.00304669e-05]


In [1129]:
# all features from time predition (basics + TAZs + PG time estimate + time of day at origin) + toll + shuffled
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.908201216631
[  9.91302155e-03   8.72775118e-01   8.39212058e-03   9.08358043e-02
   3.95848892e-03   6.99082607e-03   7.07223572e-03   6.23844564e-05]


In [1078]:
# all features from time predition + toll + airport flag
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.912034485627
[  9.28821108e-03   8.70767731e-01   8.54633583e-03   9.28378256e-02
   4.43100887e-03   6.86255277e-03   6.47258503e-03   8.54968117e-05
   5.99911075e-04   1.07562704e-04   7.78807642e-07]


In [1074]:
# all features from time predition + toll + SFO flag
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.910898216885
[  9.41266609e-03   8.70729348e-01   8.73115523e-03   9.28173735e-02
   4.10531820e-03   7.10523191e-03   6.26201635e-03   7.89886687e-05
   6.42022135e-04   1.15879675e-04]


In [1066]:
# all features from time predition (basics + TAZs + PG time estimate + time of day at origin)
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.911474355796
[ 0.00946347  0.87059879  0.0089815   0.09278085  0.00443954  0.007204
  0.00653185]


In [1062]:
# basics + TAZs + PG time estimate
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.908799860125
[ 0.87190308  0.01170751  0.09525247  0.00544769  0.00825159  0.00743765]


In [1058]:
# basics + TAZs + SFO flags
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.907454785598
[  8.73960261e-01   1.29504106e-02   9.62861669e-02   6.26096143e-03
   9.68845676e-03   7.01111570e-04   1.52631324e-04]


In [1053]:
# basics + TAZs + airport flags
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.907880848168
[  8.74006463e-01   1.30893750e-02   9.63481393e-02   6.17167293e-03
   9.57361028e-03   6.95825152e-04   1.14566292e-04   3.47929747e-07]


In [1049]:
# basics + TAZs
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.908222785837
[ 0.87403146  0.01335816  0.09649309  0.00623581  0.00988148]


In [1045]:
# basics + SFO flags only
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.907804971987
[  8.78457749e-01   1.99091840e-02   9.88086677e-02   2.46716881e-03
   3.57230012e-04]


In [1041]:
# basics + airport flags
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.907816677117
[  8.78322283e-01   1.99246051e-02   9.88041593e-02   2.70555152e-03
   2.43399427e-04   0.00000000e+00   1.51737721e-09]


In [1037]:
# basics + toll
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.905754640994
[  8.79307745e-01   2.14579277e-02   9.91397576e-02   9.45701450e-05]


In [1028]:
# smallest set of fare features
print Fmodel.oob_score_
print Fmodel.feature_importances_

0.906048630245
[ 0.87940114  0.02132278  0.09927608]


In [16]:
x = Fmodel.predict(X_fare)
rmse = np.sqrt(((x-Y)**2).mean())
rmse

1.9580733541147968

### Now try it on the test data

In [17]:
def load_and_preprocess(fname,scalar=False,shuffle=True,
                        time_columns=['to','d_pg','o_sfo','d_sfo','o_oak','d_oak',
                                      'start_TAZ','end_TAZ','toll','tdelta_pg'],
                       fare_columns=['to','d_pg','o_sfo','d_sfo','o_oak','d_oak',
                                      'start_TAZ','end_TAZ','toll','tdelta_pg']):
    
    data = load_data(fname,end_dates=False,shuffle=shuffle)

    
    ## TRAVEL TIME
    # convert to seconds since midnight
    to = (data['start_date'] - data['start_date'].apply(pd.datetools.normalize_date)).dt.seconds
    
    ## TRAVEL DIST
    # convert to meters in UTM
    myProj = Proj("+proj=utm +zone=10 +ellps=WGS84 +datum=WGS84")
    o_x,o_y = myProj(data['start_lo'].values,data['start_la'].values)
    d_x,d_y = myProj(data['end_lo'].values,data['end_la'].values)

    # compute l-2 and l-1 norm (l-1 potentially more useful as it describes driving distance if roads are all N-S...)
    d_l2 = np.sqrt((d_x-o_x)**2 + (d_y-o_y)**2)
    d_l1 = (d_x-o_x) + (d_y-o_y)


    ## AIRPORT DEPARTURES AND ARRIVALS
    # TAZ info
        # SFO: 239
        # OAK: 895
    o_sfo,d_sfo,o_oak,d_oak = data['start_TAZ']==239, data['end_TAZ']==239, data['start_TAZ']==895, data['end_TAZ']==895


    ## TOLL LIKELIHOOD
    # generate "as the crow flies" line denoting route
    route_line = gp.GeoSeries(
        [shp.geometry.LineString([(o_lo,o_la),(d_lo,d_la)]) for o_lo,o_la,d_lo,d_la in zip(
                data['start_lo'],
                data['start_la'],
                data['end_lo'],
                data['end_la'])],
        index=data.index)

    # create line separating east bay and SF
    dumbarton_bottom = [-122.06771850585938,37.46705828325743]
    dumbarton_top = [-122.16384887695312,37.54658486885501]
    sanMateo_top = [-122.28744506835938,37.69234455983933]
    bay_top = [-122.35954284667969,37.884439502831555]
    richmond_top = [-122.49886265167493,38.04134704836284]

    gg_E = [-122.420654296875,37.83131066825093]
    gg_W = [-122.94387817382812,37.68038998034691]

    dumbarton_sep = shp.geometry.LineString([dumbarton_bottom,dumbarton_top])
    sanMateo_sep = shp.geometry.LineString([dumbarton_top,sanMateo_top])
    bay_sep = shp.geometry.LineString([sanMateo_top,bay_top])
    richmond_sep = shp.geometry.LineString([bay_top,richmond_top])
    gg_sep = shp.geometry.LineString([gg_E,gg_W])

    # create indicators if route likely crossed each bridge going S/W
    toll_d = route_line.intersects(dumbarton_sep) & (data['start_lo'] > data['end_lo'])
    toll_s = route_line.intersects(sanMateo_sep) & (data['start_lo'] > data['end_lo'])
    toll_b = route_line.intersects(bay_sep) & (data['start_lo'] > data['end_lo'])
    toll_r = route_line.intersects(richmond_sep) & (data['start_lo'] > data['end_lo'])
    toll_g = route_line.intersects(gg_sep) & (data['start_la'] > data['end_la'])

    # only the bay bridge has a few toll payers (5), and a few of these look like they could have mistakes.
    # Including as a potentially useful feature
    toll = toll_b

    X = pd.DataFrame({
        'to':to,
        'd_l1':d_l1,
        'd_l2':d_l2,
        'd_pg':data['distcost'],
        'o_sfo':o_sfo,
        'd_sfo':d_sfo,
        'o_oak':o_oak,
        'd_oak':d_oak,
        'start_TAZ':data['start_TAZ'],
        'end_TAZ':data['end_TAZ'],
        'toll':toll,
        'tdelta_pg':data['timecost']
    })
    
    X_fare = X.reindex(columns=fare_columns)
    X = X.reindex(columns=time_columns)
    
    ix = X.index
    
    # scale if indicated
    if scalar:
        X = scalar.transform(X)
    
    return X,X_fare,ix

In [88]:
X_test,X_fare_test,ix = load_and_preprocess('Taxi_Query_Routed.csv',
                                            scalar=scalar,
                                            time_columns=time_cols,
                                            fare_columns=fare_cols,
                                            shuffle=False)

In [89]:
t_test = TTmodel.predict(X_test)
if scalar:
    t_scaled = scalar_t.transform(t_test)
    X_test = np.hstack((X_test,np.array([t_scaled]).T))
else:
    X_fare_test['tdelta_pred'] = t_test

In [90]:
y = Fmodel.predict(X_fare_test)
y = pd.Series(y,index = ix, name='Fare').sort_index()
y.to_csv('predictions.csv',header=True)

In [91]:
y.describe() #6

count    10000.000000
mean        17.526365
std         16.249738
min          5.384138
25%          8.645272
50%         11.404592
75%         16.583473
max        179.527540
Name: Fare, dtype: float64

In [1196]:
y.describe() #5d

count    10000.000000
mean        17.588658
std         16.391396
min          4.629940
25%          8.734614
50%         11.454243
75%         16.583563
max        185.880888
Name: Fare, dtype: float64

In [1178]:
y.describe() #5c

count    10000.000000
mean        17.588847
std         16.404879
min          4.586850
25%          8.747837
50%         11.432500
75%         16.603928
max        185.287540
Name: Fare, dtype: float64

In [1162]:
y.describe() #5b

count    10000.000000
mean        17.586787
std         16.389607
min          4.637150
25%          8.739400
50%         11.445525
75%         16.673723
max        186.358050
Name: Fare, dtype: float64

In [1149]:
y.describe() #5

count    10000.00000
mean        17.58540
std         16.51138
min          4.43500
25%          8.69500
50%         11.45500
75%         16.64275
max        189.52500
Name: Fare, dtype: float64

In [990]:
y.describe() # 4c

count    10000.000000
mean        17.634176
std         16.631098
min          4.380000
25%          8.670000
50%         11.450000
75%         16.975000
max        192.155000
Name: Fare, dtype: float64

In [862]:
y.describe() # no scaling, with 0-passenger observation left in **CURRENT SUBMISSION**

count    10000.000000
mean        17.612995
std         16.564071
min          4.435000
25%          8.655000
50%         11.482500
75%         16.866500
max        194.230000
Name: Fare, dtype: float64

#### Generate heat map of profitability

In [1204]:
TT_hm = TTmodel
F_hm = Fmodel

In [1328]:
t_pred = TT_hm.predict(X_clean)
f_pred = F_hm.predict(X_clean)

X_hm = X_clean.copy()
X_hm['t_pred'] = t_pred
X_hm['f_pred'] = f_pred
X_hm.head()

,to,start_TAZ,t_pred,f_pred
ID,,,,
0,660,38,490.35,11.814958
1,1380,30,354.88,8.438467
2,2700,10,639.35,15.889667
3,2460,40,700.80,13.292750
4,4140,45,244.06,6.944375


In [1329]:
# get list of all TAZ's
# predict for all TAZ's at 8 PM (to = 72000)
TAZs = np.array([X_hm['start_TAZ'].unique()]).T
n_TAZ = TAZs.shape[0]
t = np.ones((n_TAZ,1))*72000
to_pred = np.hstack((t,TAZs))
to_pred.shape
t_pred = np.array([TT_hm.predict(to_pred)]).T
f_pred = np.array([F_hm.predict(to_pred)]).T
results = np.hstack((TAZs,t_pred,f_pred))
results.shape

(221, 3)

In [1330]:
# create GeoDataFrame that has profit rate by TAZ
hm_df = gp.GeoDataFrame(results,columns=['TAZ','travel_time','fare'])
hm_df['profit_rate']=hm_df['fare'].div(hm_df['travel_time']) * 60 # $/min
hm_df = hm_df.set_index('TAZ')
pr_rate = hm_df['profit_rate']
pr_rate.head()

TAZ
38    1.073801
30    1.290888
10    1.221343
40    0.951611
45    1.794105
Name: profit_rate, dtype: float64

In [1331]:
# Import shapefile of TAZs
taz_shp = gp.GeoDataFrame.from_file('mtc_taz1454_clipurb2002/mtc_taz1454_clipurb2002.shp')
taz_shp = taz_shp.set_index('TAZ1454').loc[:,['geometry']]

# add profit_rate data (can't merge b/c loses geoDataFrame functionality)
ix = taz_shp.index
pr_rate = pr_rate.reindex(index=ix)
taz_shp['profit_rate'] = pr_rate
taz_shp = taz_shp[taz_shp['profit_rate'].notnull()]

taz_shp_noid = taz_shp.copy()
taz_shp_noid['TAZ'] = taz_shp.index

In [1332]:
taz_shp.to_file('pr_rate.shp')

In [1324]:
test = taz_shp_noid.to_json()
with open('data.json', 'w') as outfile:
    json.dump(test, outfile)

In [1327]:
# use folium to map
center = list(coords[['start_la','start_lo']].mean())

def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

map_osm = folium.Map(location=center,zoom_start=9)
# for i in range(0,coords.shape[0]):
#     marker_data = coords.iloc[i,:]
#     map_osm.simple_marker([marker_data['start_la'],marker_data['start_lo']],popup='Start'+str(i))
#     map_osm.simple_marker([marker_data['end_la'],marker_data['end_lo']],popup='End'+str(i))
# inline_map(map_osm)


map_osm.geo_json(geo_path='data.json', data=gp, data_out='data2.json', columns=['TAZ','profit_rate'], key_on='feature.properties.TAZ')

inline_map(map_osm)

UnboundLocalError: local variable 'json_data' referenced before assignment

### Now that I have a RF regressor with the available feature space paired down by CV, let's optimize hyperparameters

### First, create model class that we can use to test various models via CV

Then things to try:
 - grid search for best RF predictor
 - try other predictors

In [248]:
# Travel Time predictor
class TaxiFarePredictor:
     
    def __init__(self, X, T, Y, TTmodel_type = 'RF', Fmodel_type = 'RF'):
        
        
        self.TTmodel_type = TTmodel_type # travel time model type
        self.Fmodel_type = Fmodel_type # fare predictor model type
        self.X, self.T, self.F = X, T, Y # features, travel time, fare
    
        # travel time model
        self.travel_time_model(model_type = self.TTmodel_type)
        
        # train fare model
        self.fare_model(model_type = self.Fmodel_type)
    
    
    def travel_time_model(self, model_type='RF'):
    
        if model_type == 'RF':
            self.TTmodel = RandomForestRegressor(random_state=0, n_estimators=50, max_depth=50)
            self.TTmodel.fit(self.X, self.T)
        else:
            print 'Unknown model type for travel times predictor.'
            
    def grid_search(self, tuned_params, model_type='RF', cv=10):
    
        if model_type == 'RF':
            self.TTmodel = GridSearchCV(RandomForestRegressor,tuned_params, cv = cv, (random_state=0, n_estimators=50, max_depth=50)
            self.TTmodel.fit(self.X, self.T)
        else:
            print 'Unknown model type for travel times predictor.'
            
    
    def travel_time(self, Xq):
    
        return self.TTmodel.predict(Xq)
    
    
    def fare_model(self, model_type='RF'):
        
        T = self.T
        X = self.X.join(T)
        
        if model_type == 'RF':
            self.Fmodel = RandomForestRegressor(random_state=0, n_estimators=50, max_depth=50)
            self.Fmodel.fit(X, self.F)  
        else:
            print 'Unknown model type for trip fare predictor.'
    
    
    def fare(self, Xq):
        
        return self.Fmodel.predict(Xq)

    
    # can take (lat, lon) pairs or address as text, via geolocator geocoding
    def TaxiFare(self, orig, dest, tstart='0:00', npax=1):
        
        if not isinstance(orig, basestring): 
            if not isinstance(dest, basestring): 
                
                o = np.array(utm.from_latlon(orig[1], orig[0])[:2])
                d = np.array(utm.from_latlon(dest[1], dest[0])[:2])
               
        else:
            
            try:
                geolocator = Nominatim()

                origin = geolocator.geocode(orig)
                destination = geolocator.geocode(dest)
    
                o = np.array(utm.from_latlon(origin.latitude, origin.longitude)[:2])
                d = np.array(utm.from_latlon(destination.latitude, destination.longitude)[:2])
            
            except:
                
                print 'Geocoding error. Try providing WGS84 coordinates instead.'
                
                return 0
            
        # euclidean distance between origin and destination
        dist = cdist([o],[d])[0][0]
    
        #print str(depart_s)
        temp = datetime.strptime('09/01/2012 '+ str(tstart), "%m/%d/%Y %H:%M")
        depart = (temp - temp.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
        
        # features for travel time prediction
        xq = np.append(np.array([float(npax), depart, dist]), np.append(o, d))
                
        # predict travel time
        tt = self.travel_time(xq)
        
        # predict fare
        f = self.fare(np.append(xq, tt))
        
        return f[0], tt[0]

Results (RMSE scores):

- full dataset
 - RF, no dummifying categoricals, no scaling of feature space, no grid-search: 1.8347498789313159
   - saved as **predictions_draft3.csv**
 - Same as above + dummifying categoricals: 1.8481538826321522 (plus takes way longer, not clear that it's necessary but will try again when I reach a final model)
 - same as original, but including passengers: 1.8538311296987737 (also looks like it doesn't help)
 - same as original + scaling: 1.8449244519369137
   - saved this as **predictions_draft2.csv**
   - tried scaling with just continuous features, didn't make much diff. I think actually scaling everything makes more sense intuitively, will first have to OneHot the categorical TAZ features.
 - everything scaled, all data included: 1.9285044525152781
 - leaving in the 0-passenger data: 1.7324440652021653
   - way better, strange, but giving it a shot **predictions_draft4.csv**
   
- started using the bad data in the test set, so rmse's go up
 - leaving in the 0-passenger data: 2.2713969401170373
   - same as last test above, **predictions_draft4.csv**
 - leaving in all data: 1.9405090348346787
   - makes it better, and with RF already using CV then maybe this is better - saving as **predictions_draft4b.csv**
 - same as draft4b, but with dummy vars for TAZ: 1.9690324617415753
 - same as draft4b, but with dummy vars for TAZ & scaling: 1.9802905995535145
   - conclusion is that I will leave in categorical vars and not scale
   
*submitted both 4 and 4b, 4 did better, so will drop bad fare data from training from now on. current submission is* **predictions_draft4.csv**

- dropping the airport flags: 2.2848759412016229
    - **predictions_draft4c.csv**
- optimizing feature space using OOB predictions in RF regressor: 2.2609827871916055
    - **predictions_draft5.csv**
- same as draft5, but using 1000 trees for forest, instead of 10: 2.0227200119801956
    - **predictions_draft5b.csv** - **CURRENT WINNER**
- same as draft5b, but shuffled. Score was a little worse, though RMSE a little better - probably insignificant: 2.0164465269603884
    - **predictions_draft5c.csv** - also should submit with next round to compare to 5b
- same as draft 5, but using 5000 trees: 2.0183348225744107
    - **predictions_draft5d.csv**

